In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [ ]:
books = pd.read_csv('/content/Books.csv')
users = pd.read_csv('/content/Ratings.csv')
ratings = pd.read_csv('/content/Users.csv')

ParserError: ignored

## PREPROCESSING

In [ ]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [ ]:
books.drop(columns=['Image-URL-S','Image-URL-M','Image-URL-L'],inplace = True)

In [ ]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
dtypes: object(5)
memory usage: 10.4+ MB


In [ ]:
dict = {'Book-Title': 'title',
        'Book-Author': 'author',
        'Year-Of-Publication': 'publication_year',
        'Publisher':'publisher'}

books.rename(columns=dict,inplace=True)

In [ ]:
books[books.isna().any(axis=1)]
books.dropna(inplace=True)

In [ ]:

#fixing the errors in the publication year column
books.loc[(books['publication_year'] == 'DK Publishing Inc') | (books['publication_year'] == 'Gallimard')]

,ISBN,title,author,publication_year,publisher
209538,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.0...
220731,2070426769,"Peuple du ciel, suivi de 'Les Bergers\"";Jean-M...",2003,Gallimard,http://images.amazon.com/images/P/2070426769.0...
221678,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.0...


In [ ]:
#dropping the rows with errors as they are very few in number
books.drop(books[books['publication_year'] == 'DK Publishing Inc'].index, inplace = True)
books.drop(books[books['publication_year'] == 'Gallimard'].index, inplace = True)

In [ ]:
#changing the publication year to int type before feeding it to the models
books['publication_year'] = books['publication_year'].astype('int')

In [ ]:
#replacing the publication with 0 input with nan values
books.loc[(books['publication_year'] == 0)] = np.nan

#replacing the nan values in the publication year with mean of the same column
books['publication_year'].fillna((books['publication_year'].mean()), inplace=True)

#converting to integer type
books['publication_year'] = books['publication_year'].astype('int')

In [ ]:
books['publication_year'].value_counts()[:10]

2002    17626
1999    17431
2001    17358
2000    17232
1998    15766
1993    15220
1997    14890
2003    14358
1996    14030
1995    13545
Name: publication_year, dtype: int64

In [ ]:
dict = {'User-ID': 'userID',
        'Age':'age'}

users.rename(columns=dict,inplace=True)

In [ ]:
users.head()

,userID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
dict = {'User-ID': 'userID',
        'Book-Rating':'book_rating'}

ratings.rename(columns=dict,inplace=True)

In [ ]:
#merging the books, users and ratings file
df1=pd.merge(books,users)
df2 = pd.merge(df1,ratings)
df2.head()

,ISBN,title,author,publication_year,publisher,userID,Book-Rating,Location,Age
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,2,0,"stockton, california, usa",18.0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,8,5,"timmins, ontario, canada",NaN
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,8,0,"timmins, ontario, canada",NaN
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,8,0,"timmins, ontario, canada",NaN
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,8,0,"timmins, ontario, canada",NaN


In [ ]:
df2.describe()

,publication_year,userID,Book-Rating,Age
count,1.017120e+06,1.017120e+06,1.017120e+06,742854.000000
mean,1.995303e+03,1.406403e+05,2.834998e+00,37.422514
std,7.363851e+00,8.050830e+04,3.853161e+00,14.091329
min,1.376000e+03,2.000000e+00,0.000000e+00,0.000000
25%,1.992000e+03,7.041500e+04,0.000000e+00,28.000000
50%,1.997000e+03,1.414340e+05,0.000000e+00,35.000000
75%,2.001000e+03,2.114260e+05,7.000000e+00,45.000000
max,2.050000e+03,2.788540e+05,1.000000e+01,244.000000


## COLLABORATIVE FILTERING

In [ ]:
#we will only keep the observations that have book_rating greater than 0
df2 = df2[df2["Book-Rating"]>0]

In [ ]:
#counting the number of times the books have been rated
count_of_books = pd.DataFrame(df2["title"].value_counts())

In [ ]:
count_of_books.sort_values("title", ascending=False)

,title
The Lovely Bones: A Novel,707
Wild Animus,581
The Da Vinci Code,494
The Secret Life of Bees,406
The Nanny Diaries: A Novel,393
...,...
"Alienista, El",1
The Death Ship: The Story of an American Sailor,1
Naked London,1
The Complete Book of Tai Chi Chuan,1


In [ ]:
#All the books that have been rated less than 100 times - we will count them as rarely read books
rare_books = count_of_books[count_of_books["title"] <= 100].index

In [ ]:
rare_books.shape

(133186,)

In [ ]:
#All the books that are not in the rare books we will count them as frequently read books
frequent_books = df2[~df2["title"].isin(rare_books)]
frequent_books.head()

,ISBN,title,author,publication_year,publisher,userID,Book-Rating,Location,Age
19,0786868716,The Five People You Meet in Heaven,Mitch Albom,2003,Hyperion,11400,9,"ottawa, ontario, canada",49.0
20,0151008116,Life of Pi,Yann Martel,2002,Harcourt,11400,6,"ottawa, ontario, canada",49.0
22,0312195516,The Red Tent (Bestselling Backlist),Anita Diamant,1998,Picador USA,11400,7,"ottawa, ontario, canada",49.0
27,0743418174,Good in Bed,Jennifer Weiner,2002,Washington Square Press,11400,8,"ottawa, ontario, canada",49.0
37,0440222656,The Horse Whisperer,Nicholas Evans,1996,Dell,11400,6,"ottawa, ontario, canada",49.0


In [ ]:
#Let's create the User-Book matrix for the frequently read books
books_user_df = frequent_books.pivot_table(index=["userID"], columns=["title"], values="Book-Rating",fill_value=0)
books_user_df.head()

title,1984,1st to Die: A Novel,2nd Chance,A Bend in the Road,"A Child Called \It\"": One Child's Courage to Survive""",A Heartbreaking Work of Staggering Genius,A Is for Alibi (Kinsey Millhone Mysteries (Paperback)),A Map of the World,A Painted House,A Prayer for Owen Meany,...,Watership Down,We Were the Mulvaneys,When the Wind Blows,Where the Heart Is (Oprah's Book Club (Paperback)),While I Was Gone,White Oleander : A Novel,White Oleander : A Novel (Oprah's Book Club),Wicked: The Life and Times of the Wicked Witch of the West,Wild Animus,"\O\"" Is for Outlaw"""
userID,,,,,,,,,,,,,,,,,,,,,
16,0,0.0,0.0,0.0,0,0,0,0,0.0,0,...,0,0,0.0,0,0,0.0,0,0,0,0
26,0,0.0,0.0,0.0,0,0,0,0,0.0,0,...,0,0,0.0,0,0,0.0,0,0,0,0
32,0,0.0,0.0,0.0,0,0,0,0,0.0,0,...,0,0,0.0,0,0,0.0,0,0,0,0
51,0,0.0,0.0,0.0,0,0,0,0,0.0,0,...,0,0,0.0,0,0,0.0,0,0,0,0
91,0,0.0,0.0,0.0,0,0,0,0,0.0,0,...,0,0,0.0,0,0,0.0,0,0,0,0


In [ ]:
book="To Kill a Mockingbird"

In [ ]:
book=books_user_df[book]
book.sort_values(ascending=False)

userID
106007    10
128622    10
37712     10
80684     10
95359     10
          ..
93312      0
93314      0
93333      0
93358      0
278846     0
Name: To Kill a Mockingbird, Length: 16349, dtype: int64

In [ ]:
books_user_df.corrwith(book,method='pearson').sort_values(ascending=False)[:10]

title
To Kill a Mockingbird                                            1.000000
Lord of the Flies                                                0.081590
The Catcher in the Rye                                           0.080984
The Color Purple                                                 0.072714
Brave New World                                                  0.070907
Fried Green Tomatoes at the Whistle Stop Cafe                    0.062260
Snow Falling on Cedars                                           0.058128
Animal Farm                                                      0.057240
The Color of Water: A Black Man's Tribute to His White Mother    0.055991
Harry Potter and the Chamber of Secrets (Book 2)                 0.054521
dtype: float64

In [ ]:
recommend_books=books_user_df.corrwith(book).sort_values(ascending=False)[:10]
rec_books_list=list(recommend_books.index)

rec_books_list

['To Kill a Mockingbird',
 'Lord of the Flies',
 'The Catcher in the Rye',
 'The Color Purple',
 'Brave New World',
 'Fried Green Tomatoes at the Whistle Stop Cafe',
 'Snow Falling on Cedars',
 'Animal Farm',
 "The Color of Water: A Black Man's Tribute to His White Mother",
 'Harry Potter and the Chamber of Secrets (Book 2)']

## APRIORI ALGORITHM

In [ ]:
basket_Rating10 = (frequent_books[frequent_books['Book-Rating'] ==10] 
          .groupby(['title', 'author','ISBN'])['userID'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('author')) 

basket_Rating9 = (df2[df2['Book-Rating'] ==9] 
          .groupby(['title', 'author','ISBN'])['userID'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('author')) 
basket_Rating8 = (df2[df2['Book-Rating'] ==8] 
          .groupby(['title', 'author','ISBN'])['userID'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('author')) 
basket_Rating7 = (df2[df2['Book-Rating'] ==7] 
          .groupby(['title', 'author','ISBN'])['userID'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('author')) 
basket_Rating6 = (df2[df2['Book-Rating'] ==6] 
          .groupby(['title', 'author','ISBN'])['userID'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('author')) 
        

In [ ]:
 basket_Rating9.head

In [ ]:
#Specifying the rules
def one_hot_encode(x):
    if x>=1:
        return 1
    else:
        return 0

In [ ]:
# Encoding the datasets 
basket_encoded = basket_Rating10.applymap(one_hot_encode) 
basket_Rating10 = basket_encoded 

basket_encoded = basket_Rating9.applymap(one_hot_encode) 
basket_Rating9 = basket_encoded 
  
basket_encoded = basket_Rating8.applymap(one_hot_encode) 
basket_Rating8 = basket_encoded 
  
basket_encoded = basket_Rating7.applymap(one_hot_encode) 
basket_Rating7 = basket_encoded

basket_encoded = basket_Rating6.applymap(one_hot_encode) 
basket_Rating6 = basket_encoded 


In [ ]:
basket_Rating10.tail(5)

In [ ]:
from mlxtend.frequent_patterns import apriori,association_rules
# Building the model 
frq_items = apriori(basket_Rating10, min_support = 0.02, use_colnames = True) 

In [ ]:
frq_items = apriori(basket_Rating10, min_support = 0.02, use_colnames = True) 
rules = association_rules(frq_items, min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
print(rules.head()) 


## KNN ALGORITHM

In [ ]:
frequent_books.head()

,ISBN,title,author,publication_year,publisher,userID,Book-Rating,Location,Age
19,0786868716,The Five People You Meet in Heaven,Mitch Albom,2003,Hyperion,11400,9,"ottawa, ontario, canada",49.0
20,0151008116,Life of Pi,Yann Martel,2002,Harcourt,11400,6,"ottawa, ontario, canada",49.0
22,0312195516,The Red Tent (Bestselling Backlist),Anita Diamant,1998,Picador USA,11400,7,"ottawa, ontario, canada",49.0
27,0743418174,Good in Bed,Jennifer Weiner,2002,Washington Square Press,11400,8,"ottawa, ontario, canada",49.0
37,0440222656,The Horse Whisperer,Nicholas Evans,1996,Dell,11400,6,"ottawa, ontario, canada",49.0


In [ ]:
combined = frequent_books.merge(users, left_on = 'userID', right_on = 'userID', how = 'left')

us_canada_user_rating = combined[combined['Location'].str.contains("usa|canada")]
us_canada_user_rating=us_canada_user_rating.drop('Age', axis=1)
us_canada_user_rating.head()

,ISBN_x,title,author,publication_year,publisher,userID,Book-Rating_x,Location,ISBN_y,Book-Rating_y
0,0786868716,The Five People You Meet in Heaven,Mitch Albom,2003,Hyperion,11400,9,"ottawa, ontario, canada",0002005018,0
1,0786868716,The Five People You Meet in Heaven,Mitch Albom,2003,Hyperion,11400,9,"ottawa, ontario, canada",0060084405,0
2,0786868716,The Five People You Meet in Heaven,Mitch Albom,2003,Hyperion,11400,9,"ottawa, ontario, canada",0060175532,0
3,0786868716,The Five People You Meet in Heaven,Mitch Albom,2003,Hyperion,11400,9,"ottawa, ontario, canada",0060512822,0
4,0786868716,The Five People You Meet in Heaven,Mitch Albom,2003,Hyperion,11400,9,"ottawa, ontario, canada",0060958022,10


In [ ]:
from scipy.sparse import csr_matrix
us_canada_user_rating = us_canada_user_rating.drop_duplicates(['userID', 'title'])
us_canada_user_rating_pivot = us_canada_user_rating.pivot(index = 'title', columns = 'userID', values = 'Book-Rating_x').fillna(0)
us_canada_user_rating_matrix = csr_matrix(us_canada_user_rating_pivot.values)

from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(us_canada_user_rating_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
frequent_books.head()

,ISBN,title,author,publication_year,publisher,userID,Book-Rating,Location,Age
19,0786868716,The Five People You Meet in Heaven,Mitch Albom,2003,Hyperion,11400,9,"ottawa, ontario, canada",49.0
20,0151008116,Life of Pi,Yann Martel,2002,Harcourt,11400,6,"ottawa, ontario, canada",49.0
22,0312195516,The Red Tent (Bestselling Backlist),Anita Diamant,1998,Picador USA,11400,7,"ottawa, ontario, canada",49.0
27,0743418174,Good in Bed,Jennifer Weiner,2002,Washington Square Press,11400,8,"ottawa, ontario, canada",49.0
37,0440222656,The Horse Whisperer,Nicholas Evans,1996,Dell,11400,6,"ottawa, ontario, canada",49.0


In [ ]:

query_index = np.random.choice(us_canada_user_rating_pivot.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(us_canada_user_rating_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

132


In [ ]:
us_canada_user_rating_pivot.iloc[query_index,:].values.reshape(1,-1)

array([[0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
us_canada_user_rating_pivot.index[query_index]

'The Giver (21st Century Reference)'

In [ ]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(us_canada_user_rating_pivot.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, us_canada_user_rating_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for The Giver (21st Century Reference):

1: Lord of the Flies, with distance of 0.9419815974159994:
2: The Hitchhiker's Guide to the Galaxy, with distance of 0.9452464813388671:
3: A Is for Alibi (Kinsey Millhone Mysteries (Paperback)), with distance of 0.9471254591297937:
4: The Color Purple, with distance of 0.9520454114271957:
5: The Poisonwood Bible, with distance of 0.9521177103458555:


## SVD

In [ ]:
frequent_books

,ISBN,title,author,publication_year,publisher,userID,Book-Rating,Location,Age
19,0786868716,The Five People You Meet in Heaven,Mitch Albom,2003,Hyperion,11400,9,"ottawa, ontario, canada",49.0
20,0151008116,Life of Pi,Yann Martel,2002,Harcourt,11400,6,"ottawa, ontario, canada",49.0
22,0312195516,The Red Tent (Bestselling Backlist),Anita Diamant,1998,Picador USA,11400,7,"ottawa, ontario, canada",49.0
27,0743418174,Good in Bed,Jennifer Weiner,2002,Washington Square Press,11400,8,"ottawa, ontario, canada",49.0
37,0440222656,The Horse Whisperer,Nicholas Evans,1996,Dell,11400,6,"ottawa, ontario, canada",49.0
...,...,...,...,...,...,...,...,...,...
1013539,0553455710,The Street Lawyer,John Grisham,1999,Random House Audio Publishing Group,189232,9,"burlington, new jersey, usa",NaN
1013541,0553712527,A Painted House,John Grisham,2001,Random House Audio Publishing Group,189232,7,"burlington, new jersey, usa",NaN
1013542,0553712527,A Painted House,John Grisham,2001,Random House Audio Publishing Group,115338,7,"mount laurel, ,",NaN
1014420,1570429669,The Rescue,Nicholas Sparks,2000,Time Warner Audio Major,126363,9,"jupiter, florida, usa",NaN


In [ ]:
ratings

,userID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


In [ ]:
books

,ISBN,title,author,publication_year,publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company
...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm)
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press


In [ ]:
users

,userID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [ ]:
#Pivoting table to cerate a user-item matrix
user_books= frequent_books.pivot_table(index = 'ISBN', columns= 'title', values = 'Book-Rating')

In [ ]:
corr_matrix  = user_books.corr()

In [ ]:

corr_matrix.head()

title,1984,1st to Die: A Novel,2nd Chance,A Bend in the Road,"A Child Called \It\"": One Child's Courage to Survive""",A Heartbreaking Work of Staggering Genius,A Is for Alibi (Kinsey Millhone Mysteries (Paperback)),A Map of the World,A Painted House,A Prayer for Owen Meany,...,Watership Down,We Were the Mulvaneys,When the Wind Blows,Where the Heart Is (Oprah's Book Club (Paperback)),While I Was Gone,White Oleander : A Novel,White Oleander : A Novel (Oprah's Book Club),Wicked: The Life and Times of the Wicked Witch of the West,Wild Animus,"\O\"" Is for Outlaw"""
title,,,,,,,,,,,,,,,,,,,,,
1984,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A Bend in the Road,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"A Child Called \It\"": One Child's Courage to Survive""",NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
corr_matrix.loc['The Horse Whisperer'].sort_values(ascending = False)

title
The Horse Whisperer                                           1.0
1984                                                          NaN
1st to Die: A Novel                                           NaN
2nd Chance                                                    NaN
A Bend in the Road                                            NaN
                                                             ... 
White Oleander : A Novel                                      NaN
White Oleander : A Novel (Oprah's Book Club)                  NaN
Wicked: The Life and Times of the Wicked Witch of the West    NaN
Wild Animus                                                   NaN
\O\" Is for Outlaw"                                           NaN
Name: The Horse Whisperer, Length: 191, dtype: float64

In [ ]:
frequent_books[frequent_books['title'] == "The Horse Whisperer"]

,ISBN,title,author,publication_year,publisher,userID,Book-Rating,Location,Age
37,0440222656,The Horse Whisperer,Nicholas Evans,1996,Dell,11400,6,"ottawa, ontario, canada",49.0
1827,0440222656,The Horse Whisperer,Nicholas Evans,1996,Dell,11676,1,"n/a, n/a, n/a",NaN
35358,0440222656,The Horse Whisperer,Nicholas Evans,1996,Dell,14521,10,"providence, rhode island, usa",NaN
36222,0440222656,The Horse Whisperer,Nicholas Evans,1996,Dell,16795,9,"mechanicsville, maryland, usa",47.0
61389,0440222656,The Horse Whisperer,Nicholas Evans,1996,Dell,69232,10,"mooresville, north carolina, usa",55.0
...,...,...,...,...,...,...,...,...,...
969680,0385315236,The Horse Whisperer,Nicholas Evans,1995,Delacorte Press,158970,8,"richmond, british columbia, canada",NaN
969682,0385315236,The Horse Whisperer,Nicholas Evans,1995,Delacorte Press,167716,10,"rex, georgia, usa",50.0
969686,0385315236,The Horse Whisperer,Nicholas Evans,1995,Delacorte Press,218872,7,"groton, connecticut, usa",NaN
969687,0385315236,The Horse Whisperer,Nicholas Evans,1995,Delacorte Press,230377,7,"cheyenne, wyoming, usa",40.0


In [ ]:
corr_matrix = user_books.corr(method = 'pearson', min_periods = 90)

In [ ]:
penalties = {6:-10,7:0}

In [ ]:
#creating a function to fetch the user
def fetch_user(dataframe, index):
    
    return dataframe.iloc[index].dropna()

#getting the recommendations
def get_recommendations(user):
    recommendations = pd.Series()
    for i in range(len(user)):
        similarity = corr_matrix[user.index[i]].dropna()
        similarity = similarity.map(lambda x: (x * user[i]** 2) + (5 * penalties[user[i]]))
        recommendations = recommendations.append(similarity)
        
    recommendations = recommendations.groupby(recommendations.index).sum()
    recommendations.sort_values(ascending = False, inplace= True)
    return recommendations

In [ ]:
#function to get the movie recommendations by inputting the user removing movies already watched by the user
def recommender(dataframe, index):
    user = fetch_user(dataframe, index)
    movie_recommend = get_recommendations(user)
    for movie in movie_recommend.index:
        if movie in user.index:
            movie_recommend.drop(movie, inplace = True)
    return movie_recommend

In [ ]:
frequent_books.head(5)

,ISBN,title,author,publication_year,publisher,userID,Book-Rating,Location,Age
19,0786868716,The Five People You Meet in Heaven,Mitch Albom,2003,Hyperion,11400,9,"ottawa, ontario, canada",49.0
20,0151008116,Life of Pi,Yann Martel,2002,Harcourt,11400,6,"ottawa, ontario, canada",49.0
22,0312195516,The Red Tent (Bestselling Backlist),Anita Diamant,1998,Picador USA,11400,7,"ottawa, ontario, canada",49.0
27,0743418174,Good in Bed,Jennifer Weiner,2002,Washington Square Press,11400,8,"ottawa, ontario, canada",49.0
37,0440222656,The Horse Whisperer,Nicholas Evans,1996,Dell,11400,6,"ottawa, ontario, canada",49.0


In [ ]:

#creating a new user based on his love for adventure and fantasy movies and hatred for rom-coms
user_book = pd.DataFrame({'The Horse Whisperer': 1, 'The Five People You Meet in Heaven' : 9,
                        'Good in Bed' : 8, 'The Red Tent (Bestselling Backlist)': 7, 'Life of Pi': 6,
                        'A Painted House': 7, 'The Rescue': 9
                        }, index = [0])
user_book.index.name = 'user'

In [ ]:
user_book

,The Horse Whisperer,The Five People You Meet in Heaven,Good in Bed,The Red Tent (Bestselling Backlist),Life of Pi,A Painted House,The Rescue
user,,,,,,,
0,1,9,8,7,6,7,9


In [ ]:
recommender(user_book, 0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


Series([], dtype: object)

In [ ]:
# Taking the user movie matrix and filling nans with 0s.
svd_books = user_books.fillna(0)
svd_books.head()

title,1984,1st to Die: A Novel,2nd Chance,A Bend in the Road,"A Child Called \It\"": One Child's Courage to Survive""",A Heartbreaking Work of Staggering Genius,A Is for Alibi (Kinsey Millhone Mysteries (Paperback)),A Map of the World,A Painted House,A Prayer for Owen Meany,...,Watership Down,We Were the Mulvaneys,When the Wind Blows,Where the Heart Is (Oprah's Book Club (Paperback)),While I Was Gone,White Oleander : A Novel,White Oleander : A Novel (Oprah's Book Club),Wicked: The Life and Times of the Wicked Witch of the West,Wild Animus,"\O\"" Is for Outlaw"""
ISBN,,,,,,,,,,,,,,,,,,,,,
0001047973,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060094818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060158638,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060168013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
006017322X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import scipy
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import *
from sklearn.model_selection import train_test_split
%matplotlib inline 
#Dividing each rating a user gave by the mean of each user's rating
user_means = np.array(svd_books.mean(axis = 1)).reshape(-1, 1)
svd_books = svd_books.div(svd_books.mean(axis = 1), axis = 0)


In [ ]:
user_means

array([[0.04712042],
       [0.04188482],
       [0.04712042],
       [0.04450262],
       [0.04886562],
       [0.04389851],
       [0.0460733 ],
       [0.04085758],
       [0.04068631],
       [0.04327071],
       [0.04450262],
       [0.04712042],
       [0.03664921],
       [0.04328098],
       [0.0447644 ],
       [0.04461449],
       [0.04040304],
       [0.04129581],
       [0.04112603],
       [0.04282146],
       [0.0426751 ],
       [0.04732179],
       [0.04406065],
       [0.04271948],
       [0.03977369],
       [0.04275742],
       [0.04021158],
       [0.04121359],
       [0.04327726],
       [0.03141361],
       [0.04157224],
       [0.03864584],
       [0.04188482],
       [0.04275742],
       [0.04188482],
       [0.04115765],
       [0.04188482],
       [0.04230366],
       [0.04562453],
       [0.04228372],
       [0.03859519],
       [0.04498965],
       [0.04293194],
       [0.03785743],
       [0.03590127],
       [0.04450262],
       [0.04378867],
       [0.042

In [ ]:
#getting the U, S and Vt values
svd_books_matrix = svd_books.to_numpy()
U, sigma, Vt = svds(svd_books_matrix, k = 10)

In [ ]:
#Sigma value above is outputed as an array, but we need it in the form of a diagonal matrix
sigma = np.diag(sigma)

In [ ]:
#creating predictions
predicted = np.dot(np.dot(U, sigma), Vt)
predicted_ratings = np.dot(np.dot(U, sigma), Vt) * user_means


In [ ]:
predicted_ratings

array([[ 2.27346530e-32,  8.07640070e-34, -2.57035688e-33, ...,
         6.84075096e-34,  1.03659012e-33, -4.04267506e-33],
       [ 2.14565617e-32, -7.33581341e-34, -2.92854350e-33, ...,
         1.13427988e-34, -3.24390032e-33, -4.68628098e-33],
       [ 2.36295178e-32, -9.88624087e-34, -4.55565344e-33, ...,
         9.13281740e-34, -9.99963866e-34, -4.99171124e-33],
       ...,
       [-1.57963007e-16, -2.49155276e-17, -2.49071428e-17, ...,
         1.10470425e-17,  3.84583363e-18, -6.11062649e-17],
       [-1.37593926e-31,  4.67990841e-33,  4.58536461e-33, ...,
         2.40893686e-34,  1.90309387e-34,  3.33004813e-32],
       [ 1.75248818e-32,  2.09898933e-32, -6.79866104e-33, ...,
         6.58835845e-34,  1.94221668e-33,  5.39457596e-32]])

In [ ]:
predicted_df = pd.DataFrame(predicted_ratings, columns= user_books.columns)
#Data frame index starts with 0 but original dataset starts with 1, so adding 1 to index
predicted_df.index = predicted_df.index + 1

In [ ]:
predicted_df.head()

title,1984,1st to Die: A Novel,2nd Chance,A Bend in the Road,"A Child Called \It\"": One Child's Courage to Survive""",A Heartbreaking Work of Staggering Genius,A Is for Alibi (Kinsey Millhone Mysteries (Paperback)),A Map of the World,A Painted House,A Prayer for Owen Meany,...,Watership Down,We Were the Mulvaneys,When the Wind Blows,Where the Heart Is (Oprah's Book Club (Paperback)),While I Was Gone,White Oleander : A Novel,White Oleander : A Novel (Oprah's Book Club),Wicked: The Life and Times of the Wicked Witch of the West,Wild Animus,"\O\"" Is for Outlaw"""
1,2.273465e-32,8.076401e-34,-2.570357e-33,-9.673934e-32,-2.141933e-33,2.714171e-33,-1.690560e-34,-2.279318e-33,-3.254891e-32,-9.855360e-34,...,2.655934e-36,1.160045e-33,-6.694989e-17,-1.531494e-33,-1.048340e-33,-6.377757e-34,-7.816014e-34,6.840751e-34,1.036590e-33,-4.042675e-33
2,2.145656e-32,-7.335813e-34,-2.928544e-33,5.855403e-33,-1.175842e-33,9.776813e-34,9.077422e-34,7.417625e-33,6.742964e-33,-9.006567e-34,...,-1.925720e-33,-7.501975e-34,-1.883917e-16,1.591874e-33,-7.842985e-33,-1.051151e-32,7.160457e-34,1.134280e-34,-3.243900e-33,-4.686281e-33
3,2.362952e-32,-9.886241e-34,-4.555653e-33,-4.675176e-33,-7.656076e-34,2.797091e-33,4.032478e-34,1.162884e-33,-3.738836e-33,-9.113838e-34,...,-1.417348e-33,9.865909e-34,-1.105662e-16,-3.149322e-33,-6.259270e-33,-8.422806e-33,-1.008811e-33,9.132817e-34,-9.999639e-34,-4.991711e-33
4,1.823788e-32,1.958179e-34,-3.030862e-33,-1.946777e-32,-8.800180e-35,7.464791e-34,9.814648e-35,-4.783958e-33,-1.730626e-32,-1.710786e-34,...,-8.951641e-35,3.516672e-34,-1.168605e-17,-8.631510e-34,-1.491344e-33,-1.579318e-33,-3.544595e-34,1.009091e-34,-6.355021e-34,-1.274148e-33
5,-4.521354e-17,1.929475e-18,-2.103965e-17,6.690670e-17,-3.639232e-18,2.320498e-17,2.352713e-18,3.520157e-17,2.830863e-17,-2.923208e-18,...,-1.154642e-17,1.217030e-17,-5.571391e-01,-4.324159e-17,-4.296709e-17,-5.407669e-17,-1.582922e-17,1.306029e-17,6.888129e-18,-4.199536e-18


In [ ]:
#creating function to get recommendations
def svd_recommender(df_predict, ISBN, umr, number_recomm):
    user_predicted_books = df_predict.loc[ISBN, :].sort_values(ascending = False)
    original_data = umr.loc[ISBN, :].sort_values(ascending = False)
    already_rated = user_books.loc[ISBN, :].dropna()
    unrated = list(user_books.loc[1, pd.isnull(user_books.loc[ISBN, :])].index)
    
    recommendations = df_predict.loc[ISBN][unrated]
    
    recommendations = pd.DataFrame(recommendations.sort_values(ascending = False).index[:number_recomm])
    
    return recommendations, already_rated